In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "Self-Collected-NF-CSE-CICIDS.parquet"
data = pd.read_parquet(file_name)

In [3]:
data.Label.value_counts()

Label
0    636608
1    349522
Name: count, dtype: int64

In [4]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [5]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [6]:
data.Attack.unique()

array(['DoS attacks-GoldenEye', 'Benign', 'FTP-BruteForce',
       'DoS attacks-Hulk', 'Infilteration', 'DoS attacks-SlowHTTPTest',
       'DoS attacks-Slowloris', 'SQL Injection', 'SSH-Bruteforce',
       'Brute Force -Web', 'Brute Force -XSS'], dtype=object)

In [7]:
# Scale the dataset based on your needs (machine capacity) ideally ~500k rows
data_attack = data[data['Label'] == 1].sample(frac=0.2, random_state=13)
data_benign = data[data['Label'] == 0].sample(frac=1, random_state=13)
data = pd.concat([data_attack, data_benign], axis=0)
data = data.sample(frac=0.5, random_state=13).reset_index(drop=True)
data.Label.value_counts()

Label
0    318222
1     35034
Name: count, dtype: int64

In [8]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,318222,318222,318222,318222,318222,318222,318222,318222,318222,318222,...,318222,318222,318222,318222,318222,318222,318222,318222,318222,318222
Brute Force -Web,2895,2895,2895,2895,2895,2895,2895,2895,2895,2895,...,2895,2895,2895,2895,2895,2895,2895,2895,2895,2895
Brute Force -XSS,1739,1739,1739,1739,1739,1739,1739,1739,1739,1739,...,1739,1739,1739,1739,1739,1739,1739,1739,1739,1739
DoS attacks-GoldenEye,9672,9672,9672,9672,9672,9672,9672,9672,9672,9672,...,9672,9672,9672,9672,9672,9672,9672,9672,9672,9672
DoS attacks-Hulk,5810,5810,5810,5810,5810,5810,5810,5810,5810,5810,...,5810,5810,5810,5810,5810,5810,5810,5810,5810,5810
DoS attacks-SlowHTTPTest,3355,3355,3355,3355,3355,3355,3355,3355,3355,3355,...,3355,3355,3355,3355,3355,3355,3355,3355,3355,3355
DoS attacks-Slowloris,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,...,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289
FTP-BruteForce,760,760,760,760,760,760,760,760,760,760,...,760,760,760,760,760,760,760,760,760,760
Infilteration,4967,4967,4967,4967,4967,4967,4967,4967,4967,4967,...,4967,4967,4967,4967,4967,4967,4967,4967,4967,4967


In [9]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [10]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [11]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [12]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [13]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
351545,145.82.199.44,182.129.67.88,1.965582e-08,1.822779e-08,0.000275,9.909622e-07,0.001179,3.963849e-07,6.720525e-09,6.973776e-09,...,0.000150,0.000099,1.965582e-08,1.965582e-08,1.965582e-08,1.965582e-08,-1.981924e-07,2.073847e-08,"[1.9655819145400365e-08, 1.8227790718695325e-0...",351545
137232,145.82.199.44,182.129.67.88,3.922741e-07,4.584857e-09,0.026908,9.097318e-05,0.024215,7.910712e-05,1.341225e-07,1.391767e-07,...,0.002164,0.002148,3.922741e-07,3.922741e-07,3.922741e-07,3.922741e-07,-3.955356e-06,4.138807e-07,"[3.922740972325911e-07, 4.5848566531532795e-09...",137232
17579,103.57.14.23,182.129.67.88,2.072046e-07,6.485942e-07,0.001103,1.462492e-05,0.000635,8.357095e-06,0.000000e+00,1.337563e-10,...,0.134215,0.136137,2.072046e-07,2.072046e-07,2.072046e-07,2.072046e-07,-2.089274e-06,2.186175e-07,"[2.0720461815271377e-07, 6.485941872951391e-07...",17579
55026,103.57.14.23,182.129.67.88,4.382958e-09,4.064529e-09,0.000011,1.325820e-07,0.000000,0.000000e+00,4.416093e-08,4.417976e-08,...,0.002839,0.000000,4.382958e-09,4.382958e-09,4.382958e-09,4.382958e-09,-4.419400e-08,4.624373e-09,"[4.382958322053996e-09, 4.0645290044734464e-09...",55026
336057,103.57.14.23,190.201.33.51,1.836563e-07,5.748830e-07,0.000978,1.296283e-05,0.000563,7.407332e-06,0.000000e+00,1.185552e-10,...,0.118962,0.120665,1.836563e-07,1.836563e-07,1.836563e-07,1.836563e-07,-1.851833e-06,1.937722e-07,"[1.8365631932842356e-07, 5.748830418858854e-07...",336057


In [14]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [15]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [17]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [18]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [19]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [20]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [21]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [22]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [23]:
# Convert to GPU
train_g = train_g.to('cuda')

In [24]:
dgi.load_state_dict(torch.load('best_dgi_CSE_self_collected.pkl'))

/tmp/ipykernel_5748/4165988978.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_self_collected.pkl'))


<All keys matched successfully>

In [25]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [26]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [27]:
# Convert to GPU
test_g = test_g.to('cuda')

In [28]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [29]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [30]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [31]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [32]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,0.112262,-0.012728,-0.002406,-0.005496,-0.056438,-0.043761,-0.001799,0.005966,0.032646,-0.015511,...,0.000000e+00,0.0,0.208789,0.211779,3.223335e-07,3.223335e-07,3.223335e-07,3.223335e-07,-3.250135e-06,0.000000e+00
1,0.112262,-0.012728,-0.002406,-0.005496,-0.056438,-0.043761,-0.001799,0.005966,0.032646,-0.015511,...,0.000000e+00,0.0,0.000033,0.000000,6.493298e-09,6.493298e-09,6.493298e-09,6.493298e-09,-6.547285e-08,6.850951e-09
2,0.112262,-0.012728,-0.002406,-0.005496,-0.056438,-0.043761,-0.001799,0.005966,0.032646,-0.015511,...,0.000000e+00,0.0,0.000005,0.000000,9.384474e-10,9.384474e-10,9.384474e-10,9.384474e-10,-9.462499e-09,9.901374e-10
3,0.112262,-0.012728,-0.002406,-0.005496,-0.056438,-0.043761,-0.001799,0.005966,0.032646,-0.015511,...,8.203767e-07,0.0,0.000235,0.000206,4.068060e-08,4.068060e-08,4.068060e-08,4.068060e-08,-4.101883e-07,4.292130e-08
4,0.112262,-0.012728,-0.002406,-0.005496,-0.056438,-0.043761,-0.001799,0.005966,0.032646,-0.015511,...,0.000000e+00,0.0,0.066869,0.067826,1.032336e-07,1.032336e-07,1.032336e-07,1.032336e-07,-1.040919e-06,1.089197e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211949,0.139758,-0.006983,0.014483,-0.012767,-0.083813,-0.032724,0.015911,-0.000956,0.015005,0.006791,...,0.000000e+00,0.0,0.002460,0.000000,3.798568e-09,3.798568e-09,3.798568e-09,3.798568e-09,-3.830150e-08,4.007794e-09
211950,0.139758,-0.006983,0.014483,-0.012767,-0.083813,-0.032724,0.015911,-0.000956,0.015005,0.006791,...,0.000000e+00,0.0,0.003785,0.000000,5.843926e-09,5.843926e-09,5.843926e-09,5.843926e-09,-5.892514e-08,6.165812e-09
211951,0.139758,-0.006983,0.014483,-0.012767,-0.083813,-0.032724,0.015911,-0.000956,0.015005,0.006791,...,0.000000e+00,0.0,0.003028,0.000000,4.675153e-09,4.675153e-09,4.675153e-09,4.675153e-09,-4.714024e-08,4.932662e-09
211952,0.050774,0.057694,-0.050123,0.016009,-0.052450,-0.020466,0.086168,0.135568,0.014545,0.091357,...,1.184712e-03,0.0,0.000000,0.000000,5.874716e-05,5.874716e-05,5.874716e-05,5.874716e-05,-5.923561e-04,6.198298e-05


In [33]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [34]:
from pyod.models.cblof import CBLOF
n_est = [5,6,7,9,10] # cant be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [02:21<00:00,  4.73s/it]

{'n_estimators': 10, 'con': 0.04}
0.6343688976593825
              precision    recall  f1-score   support

           0     0.9225    0.9603    0.9410    190934
           1     0.4251    0.2667    0.3278     21020

    accuracy                         0.8915    211954
   macro avg     0.6738    0.6135    0.6344    211954
weighted avg     0.8731    0.8915    0.8802    211954



In [35]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [02:32<00:00,  5.07s/it]

{'n_estimators': 7, 'con': 0.05}
0.5030082171632778
              precision    recall  f1-score   support

           0     0.9019    0.9516    0.9261    190934
           1     0.1199    0.0599    0.0799     21020

    accuracy                         0.8632    211954
   macro avg     0.5109    0.5058    0.5030    211954
weighted avg     0.8244    0.8632    0.8422    211954



In [36]:
# HBOS  Embeddings+Raw (Multimodal)

In [37]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [38]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:01<00:00,  3.37s/it]

{'n_estimators': 5, 'con': 0.04}
0.49975843276914766
              precision    recall  f1-score   support

           0     0.9010    0.8725    0.8865    190934
           1     0.1004    0.1292    0.1130     21020

    accuracy                         0.7988    211954
   macro avg     0.5007    0.5009    0.4998    211954
weighted avg     0.8216    0.7988    0.8098    211954



In [39]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:10<00:00,  3.63s/it]

{'n_estimators': 5, 'con': 0.05}
0.4829317286347846
              precision    recall  f1-score   support

           0     0.8971    0.8619    0.8791    190934
           1     0.0753    0.1022    0.0867     21020

    accuracy                         0.7865    211954
   macro avg     0.4862    0.4820    0.4829    211954
weighted avg     0.8156    0.7865    0.8006    211954



In [40]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [41]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [01:50<00:00,  3.07s/it]

{'n_estimators': 25, 'con': 0.05}
0.48336803111567683
              precision    recall  f1-score   support

           0     0.8982    0.8112    0.8525    190934
           1     0.0875    0.1645    0.1142     21020

    accuracy                         0.7471    211954
   macro avg     0.4928    0.4879    0.4834    211954
weighted avg     0.8178    0.7471    0.7793    211954



In [42]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [01:59<00:00,  3.31s/it]

{'n_estimators': 30, 'con': 0.05}
0.4863446435951943
              precision    recall  f1-score   support

           0     0.8983    0.8341    0.8650    190934
           1     0.0865    0.1426    0.1077     21020

    accuracy                         0.7655    211954
   macro avg     0.4924    0.4884    0.4863    211954
weighted avg     0.8178    0.7655    0.7899    211954



In [43]:
##  IF  Emb+Raw

In [44]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [45]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:00<00:22,  1.01it/s]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:01<00:21,  1.02it/s]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:02<00:20,  1.02it/s]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 17%|█▋        | 4/24 [00:03<00:19,  1.02it/s]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-

{'n_estimators': 20, 'con': 0.01}
0.4801634892067519
              precision    recall  f1-score   support

           0     0.9008    0.9903    0.9434    190934
           1     0.0952    0.0093    0.0169     21020

    accuracy                         0.8930    211954
   macro avg     0.4980    0.4998    0.4802    211954
weighted avg     0.8209    0.8930    0.8515    211954



In [47]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:47<00:00,  1.96s/it]

{'n_estimators': 20, 'con': 0.01}
0.47848615721466853
              precision    recall  f1-score   support

           0     0.9006    0.9899    0.9431    190934
           1     0.0764    0.0076    0.0138     21020

    accuracy                         0.8925    211954
   macro avg     0.4885    0.4987    0.4785    211954
weighted avg     0.8189    0.8925    0.8510    211954

